# Cleaning Best Buy Products 

In [2]:
import pandas as pd
import re
from fractions import Fraction

In [3]:
df = pd.read_csv("..\\..\\outputs\\Best_Buy\\product_data.csv")

In [7]:
# Função para limpar a coluna 'Review Amount'
def clean_reviews(reviews):
    if isinstance(reviews, str):
        if 'be the first to write a review' in reviews:
            return 0
        else:
            # Usar regex para remover todos os caracteres não numéricos
            reviews = re.sub(r'\D', '', reviews)
            return reviews
    return reviews
# Função para limpar a coluna 'Description'
def clean_brackets(text):
    # Adicione qualquer lógica de limpeza específica para a coluna 'Description' aqui
    return text.replace('[','').replace(']','')  # Exemplo de limpeza simples (remover espaços em branco)

# Função para remover 'cubic feet' das colunas 'Capacity' e 'Dryer Capacity'
def clean_capacity(capacity):
    if isinstance(capacity, str):
        return capacity.replace(' cubic feet', '').strip()
    return capacity

def fill_brand_from_name(row):
    if pd.isna(row['Brand']):
        row['Brand'] = row['Name'].split()[0]
    return row

# Cleans SKU, Creates OBX column
def clean_SKU(sku):
    if 'obx' in sku:
        return 'Yes', sku.replace('obx ', '').strip()
    return '', sku

def convert_to_decimal(value):
    # remove inches
    if isinstance(value, str):
        value = value.replace(' inches', '').strip()
        
        # If there is a space → there is a fraction
        if ' ' in value:
            parts = value.split()
            # convert fraction into decimal
            whole = float(parts[0])
            fraction = float(Fraction(parts[1]))
            return round(whole + fraction, 2)
        else:
            return round(float(value), 2)






df = df.dropna(axis=1, how = 'all')
df = df.dropna(axis=0, how = 'all')


df = df.apply(fill_brand_from_name, axis=1)

# df['Description'] = df['Description'].apply(clean_brackets)
# df['More Images Links'] = df['More Images Links'].apply(clean_brackets)
# df['Videos Links'] = df['Videos Links'].apply(clean_brackets)

df['Review Amount'] = df['Review Amount'].apply(clean_reviews)

df['Capacity'] = df['Capacity'].apply(clean_capacity)
df['Dryer Capacity'] = df['Dryer Capacity'].apply(clean_capacity)
df['Washer Capacity'] = df['Washer Capacity'].apply(clean_capacity)
df['Capacity'] = df['Capacity'].fillna(df['Washer Capacity'])
df['Capacity'] = df['Capacity'].fillna(df['Dryer Capacity'])

df['Product Depth'] = df['Product Depth'].apply(convert_to_decimal)
df['Product Height'] = df['Product Height'].apply(convert_to_decimal)
df['Product Width'] = df['Product Width'].apply(convert_to_decimal)


df['SKU'] = df['SKU'].astype(str)
df['SKU'] = df['SKU'].str.lower()
df[['OBX', 'SKU']] = df['SKU'].apply(lambda x: pd.Series(clean_SKU(x)))

#ASK IF:
#df['Voltage'] = df['Voltage'].fillna(df['Washer Voltage'])

df.head()

,Link,Name,SKU,Price,Five Star,Review Amount,Image Link,Description,More Images Links,Videos Links,...,Estimated Annual Operating Cost (Dryer),Door Style,Propane Conversion Kit Model Compatibility,Propane (LP) Convertible,Propane Conversion Kit Included,Payment System,Total Maximum BTU Rating,Gas Connector Included,OBX,Launch Year
0,https://www.bestbuy.com/site/ge-7-8-cu-ft-10-c...,GE - 7.8 Cu. Ft. 10-Cycle Electric Dryer - White,gfd55essnww,$944.99,4.7,661.0,https://pisces.bbystatic.com/image2/BestBuy_US...,Save time in getting your laundry ready with t...,https://pisces.bbystatic.com/image2/BestBuy_US...,https://sc.liveclicker.net/service/qr?19753057...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
1,https://www.bestbuy.com/site/ge-profile-7-4-cu...,GE Profile - 7.4 Cu. Ft. Smart Electric Dryer ...,ptd60ebprdg,$899.99,4.3,549.0,https://pisces.bbystatic.com/image2/BestBuy_US...,GE Profile provides modern design and the late...,https://pisces.bbystatic.com/image2/BestBuy_US...,https://sc.liveclicker.net/service/qr?15237496...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
2,https://www.bestbuy.com/site/samsung-4-6-cu-ft...,Samsung - 4.6 Cu. Ft. Washer with Flex Auto Di...,wh46dbh500eva3,"$1,799.99",4.2,6.0,https://pisces.bbystatic.com/image2/BestBuy_US...,The Samsung Bespoke AI Laundry Hub 4.6 cu. Ft....,https://pisces.bbystatic.com/image2/BestBuy_US...,https://sc.liveclicker.net/service/qr?17520524...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
3,https://www.bestbuy.com/site/ge-7-8-cu-ft-10-c...,GE - 7.8 Cu. Ft. 10-Cycle Electric Dryer - White,gfd55essnww,$944.99,4.7,661.0,https://pisces.bbystatic.com/image2/BestBuy_US...,Save time in getting your laundry ready with t...,https://pisces.bbystatic.com/image2/BestBuy_US...,https://sc.liveclicker.net/service/qr?19753057...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN
4,https://www.bestbuy.com/site/ge-profile-7-4-cu...,GE Profile - 7.4 Cu. Ft. Smart Electric Dryer ...,ptd60ebprdg,$899.99,4.3,549.0,https://pisces.bbystatic.com/image2/BestBuy_US...,GE Profile provides modern design and the late...,https://pisces.bbystatic.com/image2/BestBuy_US...,https://sc.liveclicker.net/service/qr?15237496...,...,NaN,NaN,NaN,NaN,NaN,NaN,NaN,NaN,,NaN


In [9]:
df.to_csv('../../outputs/product_cleaned.csv')

In [8]:
df["Product Width"]

0      28.00
1      27.00
2      27.00
3      28.00
4      27.00
       ...  
322    27.00
323    23.38
324    27.00
325    27.00
326      NaN
Name: Product Width, Length: 327, dtype: float64